In [ ]:
from bs4 import BeautifulSoup
import requests

url = "https://www.alberlet.hu/kiado_alberlet/ingatlan-tipus:lakas/megye:budapest/keres:normal/limit:50"

response = requests.get(url)
response

In [ ]:
import csv
import random
import time

# Define the URL of the website to scrape
url = 'https://www.alberlet.hu/kiado_alberlet/ingatlan-tipus:lakas/megye:budapest/keres:normal/limit:50'

# Define the pattern you want to match in href attributes
pattern = '/kiado_alberlet/budapest'

# Initialize a set to store unique href values
unique_hrefs = set()

# Initialize a list to store data
data = []

# Loop through pages until there are no more "Következő" links
page = 1
while True:
    # Construct the URL for the current page
    current_url = f'https://www.alberlet.hu/kiado_alberlet/page:{page}/ingatlan-tipus:lakas/megye:budapest/keres:normal/limit:50'


    # Send a GET request to the current page
    response = requests.get(current_url)

    # Check if the response status code indicates success
    if response.status_code == 200:
        # Parse the page content with BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find all <a> tags with href attributes matching the pattern
        matching_links = [a for a in soup.find_all('a', href=True) if pattern in a['href']]

        for link in matching_links:
            href_value = link['href']
            if href_value not in unique_hrefs:
                unique_hrefs.add(href_value)
                print("Href:", href_value)

            # Extract and print the prices of matching links
            div_col = link.find('div', class_='col')
            if div_col is not None:
                price_span = div_col.find('span', class_='price')
                if price_span is not None:
                    price = price_span.get_text(strip=True)
                    print("Price:", price)
                    data.append([href_value, price])

        # Check if there's a "Következő" link
        next_page_link = soup.find('a', class_='pager_link', title='következő')
        if next_page_link:
            # Increment the page number for the next iteration
            page += 1
        else:
            # No more pages to scrape, exit the loop
            break
    else:
        # Handle request errors, such as retries or logging
        print(f'Error: {response.status_code}')
        break
        
    delay_seconds = random.uniform(5, 10)
    print(f"Waiting for {delay_seconds:.2f} seconds before the next scrape...")
    time.sleep(delay_seconds)
    
# Save the data to a CSV file
with open('prices.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    # Write headers
    csvwriter.writerow(['Href', 'Price'])
    # Write data rows
    csvwriter.writerows(data)
